In [1]:
source = 'react'

In [2]:
import chromadb
from chromadb.utils import embedding_functions
from chromadb.config import Settings

chroma_client = chromadb.Client(Settings(
    chroma_db_impl='duckdb+parquet',
    persist_directory="./db/" # Optional, defaults to .chromadb/ in the current directory
    )
)

Using embedded DuckDB with persistence: data will be stored in: ./db/


In [3]:
from chromadb.utils import embedding_functions

In [4]:
ef = embedding_functions.InstructorEmbeddingFunction(
model_name="hkunlp/instructor-large", device="cuda")

/home/lucas/fluff/semsearch/.venv/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [5]:
import json
with open(f"../md-to-prompt/{source}.json", "r") as f:
    corpus = json.loads(f.read())

In [6]:
# corpus=corpus[0:100]

In [7]:
# corpus

In [8]:
print(len(corpus))

4970


In [9]:
# raise 1

In [10]:
try:
    chroma_client.delete_collection(name=source,)
except Exception as e:
    print(e)

list index out of range


In [11]:
collection = chroma_client.create_collection(name=source, embedding_function=ef)

In [12]:
def merge_content(it):
    path = " / ".join(it["headings"])
    c = it["content"]
    return "This document path is: " + path + ".\n\nAnd the content is: " + c
    

In [13]:
documents = [merge_content(it) for it in corpus]
ids = [str(it) for it in range(len(corpus))]

In [14]:
collection.add(
    documents=documents,
    ids=ids
)


In [15]:
# collection.create_index()

In [16]:
# chroma_client.persist()

In [17]:
collection.count()

4970

In [18]:
results = collection.query(
    query_texts=["When is data really deleted in Clickhouse with the new `DELETE` query"],
    n_results=10
)

In [19]:
results

{'ids': [['1579',
   '1725',
   '4216',
   '1727',
   '4181',
   '4221',
   '2655',
   '1563',
   '1722',
   '4180']],
 'embeddings': None,
 'documents': [['This document path is: docs/react/learn/synchronizing-with-effects.md / How to handle the Effect firing twice in development? {/how-to-handle-the-effect-firing-twice-in-development/} / Fetching data {/fetching-data/}.\n\nAnd the content is: You can\'t "undo" a network request that already happened, but your cleanup function should ensure that the fetch that\'s *not relevant anymore* does not keep affecting your application. If the `userId` changes from `\'Alice\'` to `\'Bob\'`, cleanup ensures that the `\'Alice\'` response is ignored even if it arrives after `\'Bob\'`.\n',
   'This document path is: docs/react/learn/you-might-not-need-an-effect.md / How to remove unnecessary Effects {/how-to-remove-unnecessary-effects/} / Fetching data {/fetching-data/}.\n\nAnd the content is: **To fix the race condition, you need to [add a cleanup

In [20]:
for a,b in zip(results['ids'][0], results['distances'][0]):
    idx = int(a)
    print(idx)
    print(" > ".join(corpus[idx]["headings"]))
    print(b)
    print()
    print(corpus[idx]["content"])
    print()
    print()
    print()

1579
docs/react/learn/synchronizing-with-effects.md > How to handle the Effect firing twice in development? {/how-to-handle-the-effect-firing-twice-in-development/} > Fetching data {/fetching-data/}
0.3304305076599121

You can't "undo" a network request that already happened, but your cleanup function should ensure that the fetch that's *not relevant anymore* does not keep affecting your application. If the `userId` changes from `'Alice'` to `'Bob'`, cleanup ensures that the `'Alice'` response is ignored even if it arrives after `'Bob'`.




1725
docs/react/learn/you-might-not-need-an-effect.md > How to remove unnecessary Effects {/how-to-remove-unnecessary-effects/} > Fetching data {/fetching-data/}
0.33610448241233826

**To fix the race condition, you need to [add a cleanup function](/learn/synchronizing-with-effects#fetching-data) to ignore stale responses:**
```js {5,7,9,11-13}
function SearchResults({ query }) {
  const [results, setResults] = useState([]);
  const [page, setPage]

In [21]:
results = collection.query(
    query_texts=["When is data really deleted in Clickhouse with the new `DELETE` query"],
    n_results=10
)

In [22]:
results

{'ids': [['1579',
   '1725',
   '4216',
   '1727',
   '4181',
   '4221',
   '2655',
   '1563',
   '1722',
   '4180']],
 'embeddings': None,
 'documents': [['This document path is: docs/react/learn/synchronizing-with-effects.md / How to handle the Effect firing twice in development? {/how-to-handle-the-effect-firing-twice-in-development/} / Fetching data {/fetching-data/}.\n\nAnd the content is: You can\'t "undo" a network request that already happened, but your cleanup function should ensure that the fetch that\'s *not relevant anymore* does not keep affecting your application. If the `userId` changes from `\'Alice\'` to `\'Bob\'`, cleanup ensures that the `\'Alice\'` response is ignored even if it arrives after `\'Bob\'`.\n',
   'This document path is: docs/react/learn/you-might-not-need-an-effect.md / How to remove unnecessary Effects {/how-to-remove-unnecessary-effects/} / Fetching data {/fetching-data/}.\n\nAnd the content is: **To fix the race condition, you need to [add a cleanup

In [24]:
chroma_client.persist()

True